<a href="https://colab.research.google.com/github/karumugamio/SMAIProject-Team31/blob/master/Match_and_Beam_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
import os
os.chdir('/gdrive/MyDrive/SocialComputing2/workingdir')

In [ ]:
#all prerequsites are packed into requirements.txt style
#!pip install -r requirements.txt 

In [ ]:
import sys, os
from transformers import AutoTokenizer, BertModel, GPT2Model
import en_core_web_md
from tqdm import tqdm
import json

In [ ]:
import torch
print(torch.cuda.is_available())
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

True


In [ ]:
#configurations 
language_model= 'bert-large-cased'
use_cuda = True
input_folder = 'input_through/'
output_folder = 'output_through/'
threshold = 0.006
device = "cuda:0"

In [ ]:
# Util method to handle string contents to proper Boolean
def str2bool(v):
    if isinstance(v, bool):
       return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [ ]:
nlp = en_core_web_md.load()
tokenizer = AutoTokenizer.from_pretrained(language_model)

if 'gpt2' in language_model:
    encoder = GPT2Model.from_pretrained(language_model)
else:
    encoder = BertModel.from_pretrained(language_model)

encoder = encoder.to(device)
encoder.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [ ]:
from utils import compress_attention, create_mapping, BFS, build_graph, is_word
from multiprocessing import Pool
import spacy
import en_core_web_sm
import torch
from transformers import AutoTokenizer, BertModel, GPT2Model
from constant import invalid_relations_set


def process_matrix(attentions, layer_idx = -1, head_num = 0, avg_head=False, trim=True, use_cuda=True):
    if avg_head:
        if use_cuda:
            attn =  torch.mean(attentions[0][layer_idx], 0).cpu()
        else:
            attn = torch.mean(attentions[0][layer_idx], 0)
        attention_matrix = attn.detach().numpy()
    else:
        attn = attentions[0][layer_idx][head_num]
        if use_cuda:
            attn = attn.cpu()
        attention_matrix = attn.detach().numpy()

    attention_matrix = attention_matrix[1:-1, 1:-1]

    return attention_matrix

def bfs(args):
    s, end, graph, max_size, black_list_relation = args
    return BFS(s, end, graph, max_size, black_list_relation)


def check_relations_validity(relations):
    for rel in relations:
        if rel.lower() in invalid_relations_set or rel.isnumeric():
            return False
    return True

def global_initializer(nlp_object):
    global spacy_nlp
    spacy_nlp = nlp_object

def filter_relation_sets(params):
    triplet, id2token = params

    triplet_idx = triplet[0]
    confidence = triplet[1]
    head, tail = triplet_idx[0], triplet_idx[-1]
    if head in id2token and tail in id2token:
        head = id2token[head]
        tail = id2token[tail]
        relations = [ spacy_nlp(id2token[idx])[0].lemma_  for idx in triplet_idx[1:-1] if idx in id2token ]
        if len(relations) > 0 and check_relations_validity(relations) and head.lower() not in invalid_relations_set and tail.lower() not in invalid_relations_set:
            return {'h': head, 't': tail, 'r': relations, 'c': confidence }
    return {}

def parse_sentence(sentence, tokenizer, encoder, nlp, use_cuda=True):
    '''Implement the match part of MAMA
    '''
    #print("Inside Parse Sentence")
    tokenizer_name = str(tokenizer.__str__)
    
    inputs, tokenid2word_mapping, token2id, noun_chunks  = create_mapping(sentence, return_pt=True, nlp=nlp, tokenizer=tokenizer)
    #print("After Creating Mapping")
    with torch.no_grad():
        if use_cuda:
            #print("Cuda is Enabled")
            for key in inputs.keys():
                inputs[key] = inputs[key].cuda()
        #print("Populated all Inputs")
        #print(inputs)

        outputs = encoder(**inputs, output_attentions=True)
    trim = True
    if 'GPT2' in tokenizer_name:
        trim  = False
    #print("After Trim")
    '''
    Use average of last layer attention : page 6, section 3.1.2
    '''
    attention = process_matrix(outputs[2], avg_head=True, trim=trim, use_cuda=use_cuda)
    #print("Extracted Attention")
    merged_attention = compress_attention(attention, tokenid2word_mapping)
    attn_graph = build_graph(merged_attention)

    tail_head_pairs = []
    for head in noun_chunks:
        for tail in noun_chunks:
            if head != tail:
                tail_head_pairs.append((token2id[head], token2id[tail]))

    black_list_relation = set([ token2id[n]  for n in noun_chunks ])

    all_relation_pairs = []
    id2token = { value: key for key, value in token2id.items()}

    with Pool(10) as pool:
        params = [  ( pair[0], pair[1], attn_graph, max(tokenid2word_mapping), black_list_relation, ) for pair in tail_head_pairs]
        for output in pool.imap_unordered(bfs, params):
            if len(output):
                all_relation_pairs += [ (o, id2token) for o in output ]

    triplet_text = []
    with Pool(10, global_initializer, (nlp,)) as pool:
        for triplet in pool.imap_unordered(filter_relation_sets, all_relation_pairs):
            if len(triplet) > 0:
                triplet_text.append(triplet)
    return triplet_text

/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
from os import listdir
from os.path import isfile, join
import csv
import time

overallstart = time.time()

onlyfiles = [os.path.abspath(join(input_folder, f)) for f in listdir(input_folder) if isfile(join(input_folder, f))]

print(onlyfiles)

filter_data = [x for x in onlyfiles if '.csv'in x]
print(filter_data)

file_count = 0
for ipfile in filter_data:
  sourceFileReader = csv.reader(open(ipfile))
  file_count = file_count+1
  count = 0
  for row in sourceFileReader: 
      
      persentence_start = time.time()
      count = count +1
      skipcount = 0
      print("Count::"+str(count))
      print("Start time is :"+str(persentence_start))
      src_sentence  = row[1].strip() 
      if count > skipcount:
        
        opfileName = "throughfareName_matchop_file_"+str(file_count)+"sent_"+str(count)+".jsonl"
        output_filename = os.path.abspath(join(output_folder,opfileName))
        valid_triplets = []
        src_sentence  = row[1].strip() 
        if len(src_sentence):
        
            for sent in nlp(src_sentence).sents:
                # Match
                for triplets in parse_sentence(sent.text, tokenizer, encoder, nlp, use_cuda=use_cuda):
                    valid_triplets.append(triplets)

            with open(output_filename, 'w') as op:
              op.write(json.dumps( valid_triplets )+'\n')
            
            persentence_end = time.time()
            persent_total = persentence_end-persentence_start
            print('total time taken for :['+str(count)+'] :['+str(persent_total)+']') 


overallend = time.time()

total = overallend-overallstart
print('total time taken :['+str(total)+']') 

['/gdrive/My Drive/SocialComputing2/workingdir/input_through/resolved_coref_ThoroughFares.csv']
['/gdrive/My Drive/SocialComputing2/workingdir/input_through/resolved_coref_ThoroughFares.csv']
Count::1
Start time is :1618538277.079641
total time taken for :[1] :[2.8298823833465576]
Count::2
Start time is :1618538279.9097118
total time taken for :[2] :[2.2849414348602295]
Count::3
Start time is :1618538282.1957963
total time taken for :[3] :[153.62542605400085]
Count::4
Start time is :1618538435.8215122
total time taken for :[4] :[247.91760110855103]
Count::5
Start time is :1618538683.7405672
total time taken for :[5] :[128.09070348739624]
Count::6
Start time is :1618538811.832661
total time taken for :[6] :[146.59278678894043]
Count::7
Start time is :1618538958.4262002
total time taken for :[7] :[233.51269912719727]
Count::8
Start time is :1618539191.9403355
total time taken for :[8] :[156.30293583869934]
Count::9
Start time is :1618539348.244286
total time taken for :[9] :[109.92418074

In [ ]:
"""
with open(input_filename, 'r') as f, open(output_filename, 'w') as g:
  for idx, line in enumerate(tqdm(f)):
      sentence  = line.strip() 
      if len(src_sentence):
          valid_triplets = []
          for sent in nlp(src_sentence).sents:
              # Match
              for triplets in parse_sentence(sent.text, tokenizer, encoder, nlp, use_cuda=use_cuda):
                  valid_triplets.append(triplets)


                  """

In [ ]:
len(valid_triplets)

In [ ]:
#!pip install spacy-dbpedia-spotlight

In [ ]:
"""
import spacy_dbpedia_spotlight
# a new blank model will be created, with the language code provided in the parameter
nlp = spacy_dbpedia_spotlight.create('en')
# in this case, the pipeline will be only contain the EntityLinker
print(nlp.pipe_names)

"""

In [ ]:
"""
import spacy
nlp = spacy.blank('en')
nlp.add_pipe('dbpedia_spotlight')

doc = nlp('Prince Charles Died couple of day back in England inside his palace.')
print([(ent.text, ent.kb_id_, ent._.dbpedia_raw_result['@types']) for ent in doc.ents])

for ent in doc.ents:
    print(ent._.dbpedia_raw_result)
"""
